# Author : ASSAZZIN [ KSOURI Azer ]

# SETUP

In [ ]:
!nvidia-smi 

Mon Feb 21 07:36:57 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    25W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%%capture
!pip install gdown -q
!pip install efficientnet_pytorch -q
!pip uninstall albumentations --y
!pip install albumentations==1.0.3 -q 
!pip install timm==0.4.12 -q
!pip install yacs==0.1.8 pyyaml==5.4.1 -q

In [ ]:
%%capture
!pip uninstall opencv-python --y
!pip install --upgrade opencv-python
!pip install --upgrade opencv-contrib-python

In [ ]:
! pip install kaggle -q
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d ahmedmesslmani01/task-mate-kenyan-sign-language-classification
!unzip -q  /content/task-mate-kenyan-sign-language-classification.zip

100% 1.10G/1.10G [00:16<00:00, 40.3MB/s]
100% 1.10G/1.10G [00:16<00:00, 71.0MB/s]


In [ ]:
exit(0) # if you run on colab exit(0) will restart the session so packages versions will be updated

# IMPORTS

In [ ]:
import pandas as pd 
import numpy as np
import random
import os
import cv2
import io
from tqdm import tqdm_notebook as tqdm 
import seaborn as sns
import matplotlib.pyplot as plt

import sklearn
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score ,roc_curve, auc , f1_score , precision_score , log_loss
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import torchvision
import torchvision.models as models
from   torchvision import datasets, transforms, models
from   torchvision import transforms

import timm
import torch
import torch.nn as nn
from   torch.utils.data import DataLoader, Dataset
from   torch.optim.lr_scheduler import MultiStepLR
from   torch.optim.lr_scheduler import OneCycleLR
from   torch.nn import functional as F
from   torch.autograd import Variable

import albumentations
import albumentations as A
import albumentations.augmentations.transforms as AT
from   albumentations.pytorch.transforms import ToTensorV2
import albumentations.augmentations.transforms as AT
import albumentations.augmentations.geometric.transforms as AG
import albumentations.augmentations.geometric.rotate as AGR
from albumentations import (
    HorizontalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine,
    IAASharpen, IAAEmboss, RandomContrast, RandomBrightness, Flip, OneOf, Compose, RandomGamma, ElasticTransform, ChannelShuffle,RGBShift, Rotate
)

from efficientnet_pytorch import EfficientNet

import yaml
from yacs.config import CfgNode as CN
from functools import reduce , wraps

import warnings
warnings.simplefilter('ignore')

# Data Preparation

In [ ]:
Train = pd.read_csv('Train.csv')
Train.head()

,img_IDS,Label
0,ImageID_33HMDMJ3,Temple
1,ImageID_V5POEX4O,Church
2,ImageID_89CCCCW6,Enough/Satisfied
3,ImageID_GNJXOWX5,Me
4,ImageID_7Q9LOP7R,Love


In [ ]:
targets = Train.Label.unique().tolist()

Target_Mapper = dict(zip(targets,[i for i in range(len(targets))]))
InverseTarget_Mapper = dict(zip([i for i in range(len(targets))],targets))

In [ ]:
Train.Label = Train.Label.map(Target_Mapper)

In [ ]:
Test = pd.read_csv('Test.csv')

# Config

In [ ]:
_C = CN()

_C.data = CN()
_C.data.data_path = 'Images' # path of the folder that contains train and test images
_C.data.Image_ID_col = 'img_IDS'
_C.data.LABELS = 'Label'

_C.preprocess=CN()
_C.preprocess.input_shape = [384,384]

_C.model = CN()
_C.model.name_model = "swin_base_patch4_window12_384"

_C.model.train_bs = 12
_C.model.test_bs = 32

_C.model.base_lr = 5e-5
_C.model.weight_decay = 1e-6
_C.model.epochs = 7

_C.n_folds = 10
_C.num_classes = 9
_C.seed = 42
_C.metric = 'LogLoss'
_C.device = "cuda" if torch.cuda.is_available() else "cpu"

def get_cfg_defaults():
    """Get a yacs CfgNode object with default values for my_project."""
    # Return a clone so that the defaults will not be altered
    # This is for the "local variable" use pattern
    #return _C.clone()
    return _C

def dump_cfg(config = get_cfg_defaults() , path = "experiment.yaml"):
    """Save a yacs CfgNode object in a yaml file in path."""
    stream = open(path, 'w')
    stream.write(config.dump())
    stream.close()

def inject_config(funct):
    """Inject a yacs CfgNode object in a function as first arg."""
    @wraps(funct)
    def function_wrapper(*args,**kwargs):
        return funct(*args,**kwargs,config=_C)  
    return function_wrapper

def dump_dict(config,path="config.yaml"):
        stream = open(path, 'w')
        yaml.dump(config,stream)
        stream.close()

c=get_cfg_defaults()

# Utils

In [ ]:
class ImageDataset(Dataset):
    @inject_config
    def __init__(self,df, transform=None,mode='train',config =CN):
        self.df = df
        self.transform = transform
        self.mode = mode
        self.dir = config.data.data_path
        self.Image_ID_col = config.data.Image_ID_col
        self.LABELS = config.data.LABELS

    def cv_reader(self,path):
        img = cv2.imread(path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        return img

    def __getitem__(self, index):
        image_name = os.path.join(self.dir,f"{self.df[self.Image_ID_col][index]}.jpg")
        image = self.cv_reader(image_name)
        if self.transform is not None:
          image = self.transform(image=image)
          image=image["image"]

        if self.mode == 'train':
            label = self.df[self.LABELS][index]
            return {'image' : torch.tensor(image,dtype=torch.float), 
                    'label' : torch.tensor(label,dtype = torch.float) }
        return {'image' : torch.tensor(image,dtype=torch.float)}
        
    def __len__(self):
        return self.df.shape[0]

In [ ]:
class TimmModel(nn.Module):
    @inject_config
    def __init__(self,model_name,config:CN):
        super().__init__()
        if 'resne' in model_name : 
          self.model = timm.create_model(model_name, pretrained=True)
          n_features = self.model.fc.in_features
          self.model.global_pool = nn.Identity()
          self.model.fc  = nn.Identity()
          self.pooling = nn.AdaptiveAvgPool2d(1)  
          self.fc  = nn.Linear(n_features,config.num_classes)  
        elif 'efficient' in model_name :
          self.model = timm.create_model(model_name, pretrained=True)
          n_features = self.model.classifier.in_features
          self.model.global_pool = nn.Identity()
          self.model.classifier = nn.Identity()
          self.pooling = nn.AdaptiveAvgPool2d(1)
          self.fc  = nn.Linear(n_features,config.num_classes)
            
    def forward(self, x):
        bs = x.size(0)
        features = self.model(x)
        pooled_features = self.pooling(features).view(bs, -1)
        output = self.fc(pooled_features)
        return output

In [ ]:
class SwinModel(nn.Module):
    @inject_config
    def __init__(self,model_name,config:CN):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=True)
        n_features = self.model.head.in_features
        self.model.head  = nn.Linear(n_features,config.num_classes)
            
    def forward(self, x):
        output = self.model(x)
        return output

In [ ]:
class TrochvisionModel(nn.Module):
    @inject_config
    def __init__(self,model_name,config:CN):
        super().__init__()
        self.model = getattr(models, model_name)(True)
        n_features = self.model.fc.in_features
        self.model.fc = nn.Linear(n_features,config.num_classes)
        
    def forward(self, x):
        pooled_features = self.model(x)
        return  pooled_features

In [ ]:
class AverageMeter():
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()
    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0
    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
class Trainer :
  @inject_config
  def __init__(self,config : CN) :
    self.device = config.device

  def get_score(self,y_true, y_pred):
    return f1_score(y_true, np.argmax(y_pred,axis=1),average='weighted')

  def loss_fn(self,outputs,targets):
    criterion = nn.CrossEntropyLoss()
    loss = criterion(outputs,targets)
    return loss
  
  def train_fn(self,train_data_loader,model,optimizer,scheduler = None):
    model.train()
    losses = AverageMeter()
    roc_auc = AverageMeter()
    tk0 = tqdm(train_data_loader, total=len(train_data_loader))
    tot_loss = 0
    for bi,d in enumerate(tk0):
      images =  d['image']
      labels =  d['label']
      #send them to device 
      images  = images.to(self.device,dtype=torch.float)
      labels  = labels.to(self.device,dtype=torch.long)

      optimizer.zero_grad()    
      outputs  = model(images)
      loss = self.loss_fn(outputs,labels)
      loss.backward()
      optimizer.step()
      tot_loss = tot_loss + loss.item()
      losses.update(loss.item(), labels.size(0))
      tk0.set_postfix(loss=losses.avg)
      if scheduler is not None:
        scheduler.step()

    loss_score = tot_loss/len(train_data_loader)
    f1_scoree = self.get_score(labels.cpu().numpy(), torch.nn.functional.softmax(outputs).cpu().detach().numpy())
    
    print("Training loss for this epoch: ", loss_score)
    print("Training F1 Score for this epoch: ", f1_scoree)
    return f1_scoree

  def eval_fn(self,valid_data_loader,model):
    model.eval()
    tot_loss = 0
    final_outputs = []
    final_targets = []
    with torch.no_grad():
      for bi,d in enumerate(valid_data_loader):
        images = d['image']
        labels = d['label']
        #send them to device 
        images = images.to(self.device,dtype=torch.float)
        labels = labels.to(self.device,dtype=torch.long)
        
        outputs  = model(images)
        loss = self.loss_fn(outputs,labels)
        tot_loss = tot_loss + loss.item()

        final_outputs.append(torch.nn.functional.softmax(outputs).cpu().detach().numpy())
        final_targets.append(labels.cpu().numpy())

      final_targets = np.concatenate(final_targets)
      final_outputs = np.concatenate(final_outputs)
      
      loss_score  = tot_loss/len(valid_data_loader)
      f1_scoree = self.get_score(final_targets,final_outputs)
      print("Validation loss for this epoch: ",loss_score)
      print('Validation F1 Score for this epoch',f1_scoree)
    return loss_score,f1_scoree , final_outputs

In [ ]:
class RandomCenterCrop:
    def __init__(self,height_limit , width_limit):
        if isinstance(height_limit, (int, float)):
          self.height_limit = [height_limit , 1]
        else :
          self.height_limit = height_limit

        if isinstance(width_limit, (int, float)):
          self.width_limit = [width_limit , 1]
        else :
          self.width_limit = width_limit

    def __call__(self,image,**kwargs):
        #print("image hey : ",image.shape)
        height = int(random.uniform(self.height_limit[0] , self.height_limit[1]) * image.shape[0])
        width = int(random.uniform(self.width_limit[0] , self.width_limit[1]) * image.shape[1])
        return A.CenterCrop(height , width,p=1)(image=image)["image"]

class RandomCenterBlur:
    def __init__(self,height_limit , width_limit, blur_limit):
        if isinstance(height_limit, (int, float)):
          self.height_limit = [height_limit , 1]
        else :
          self.height_limit = height_limit

        if isinstance(width_limit, (int, float)):
          self.width_limit = [width_limit , 1]
        else :
          self.width_limit = width_limit
        
        if isinstance(blur_limit, (int, float)):
          self.blur_limit = [blur_limit , 199]
        else :
          self.blur_limit = blur_limit

    def __call__(self,img,**kwargs):
      y,x,_ = img.shape
      height = int(random.uniform(self.height_limit[0] , self.height_limit[1]) * img.shape[0])
      width = int(random.uniform(self.width_limit[0] , self.width_limit[1]) * img.shape[1])
      startx = (x//2)-(width//2)
      starty = (y//2)-(height//2)
      center_img = img[starty:starty+height,startx:startx+width]
      blured = A.Compose([A.GaussianBlur(blur_limit=self.blur_limit,always_apply=True ,p=1)])(image=center_img)['image']

      # Insert ROI back into image
      img[starty:starty+height,startx:startx+width] = blured
      return img

In [ ]:
class Augmentation :
  @inject_config
  def __init__(self,config :CN) :
    self.input_shape = config.preprocess.input_shape
    self.SEED_VAL  = config.seed
    
  def seed_all(self):
        random.seed(self.SEED_VAL)
        np.random.seed(self.SEED_VAL)
        torch.manual_seed(self.SEED_VAL)
        torch.cuda.manual_seed_all(self.SEED_VAL)
        os.environ['PYTHONHASHSEED'] = str(self.SEED_VAL)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        
  def train_transform(self,) :
    self.seed_all()
    train_transform = A.Compose([
                              A.OneOf([
                                    A.RandomResizedCrop (self.input_shape[0], self.input_shape[0], scale=(0.8, 1.0), ratio=(0.8, 1.0), p=0.5),       
                                    A.Resize(height=self.input_shape[0], width=self.input_shape[1], p=0.5),
                                  ]
                                  , p=1),
                              OneOf([
                                     A.Lambda(name = "RandomCenterBlur" , 
                                    image= RandomCenterBlur(height_limit=[0.5 , 0.7] , width_limit=[0.7 , 0.9],blur_limit=(101,199)) ,
                                    p=0.5)  , 
                                    #  MotionBlur(p=.2),
                                    #  MedianBlur(blur_limit=3, p=.1),
                                     Blur(blur_limit=15, p=.1),
                              ], p=0.3),
                              OneOf([CLAHE(clip_limit=4,p=0.5),
                                     A.Equalize(p=0.3),
                                     A.HueSaturationValue(p=0.3),
                                     A.ColorJitter(p=0.3),
                                    ], p=0.3),
                              OneOf([
                                     IAASharpen(p=0.1),
                                    #  IAAEmboss(p=0.5),
                                     RandomContrast(p=0.5),
                                     RandomBrightness(p=0.5),
                              ], p=0.3),
                              OneOf([ 
                                  A.Transpose(p=0.05),
                                  A.VerticalFlip(0.01),
                                  # RandomRotate90(p=0.2),
                              ], p=0.05),
                              A.HorizontalFlip(p=0.3),
                              A.Rotate (limit=45, interpolation=1, border_mode=4, value=None, mask_value=None, p=0.3),
                              A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), p=1.0),
                              ToTensorV2(),
                              ])
    return train_transform
  
  def test_trasnform(self,Augstype='Resize') :
    if   Augstype =='RandomCrop' : 
        test_transform = A.Compose([ 
                                  A.RandomResizedCrop (self.input_shape[0], self.input_shape[0], scale=(0.8, 1.0), ratio=(0.8, 1.0), p=1.0),
                                  A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), p=1.0),
                                  ToTensorV2(),
                                  ])
    elif Augstype == 'Center_Crop_Resize' :
        test_transform = A.Compose([ 
                                  A.Lambda(name = "RandomCenterCrop" , image = RandomCenterCrop( height_limit=[0.7 , 0.9] , width_limit=[0.95 , 1] ) , p = 1.0),
                                  A.Resize(height=self.input_shape[0], width=self.input_shape[1],p=1),
                                  A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), p=1.0),
                                  ToTensorV2(),
                                  ])
    elif Augstype == 'Resize' :
        test_transform = A.Compose([ 
                                  A.Resize(height=self.input_shape[0], width=self.input_shape[1],p=1),
                                  A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), p=1.0),
                                  ToTensorV2(),
                                  ])
    elif Augstype == 'HueFilpResize' :
        test_transform = A.Compose([ A.HueSaturationValue(p=1.0),
                                  A.Resize(height=self.input_shape[0], width=self.input_shape[1],p=1),
                                  A.HorizontalFlip(p=1.0),  
                                  A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), p=1.0),
                                  ToTensorV2(),
                                  ])
    elif Augstype == 'RRR_PROCESS' :
        test_transform = A.Compose([ 
                                  A.OneOf([
                                    A.RandomResizedCrop (self.input_shape[0], self.input_shape[0], scale=(0.8, 1.0), ratio=(0.8, 1.0), p=0.8),       
                                    A.Resize(height=self.input_shape[0], width=self.input_shape[1], p=0.2),
                                  ]
                                  , p=1),
                                  OneOf([CLAHE(clip_limit=2),
                                     IAASharpen(),
                                     IAAEmboss(),
                                     RandomContrast(),
                                     RandomBrightness(),
                                  ], p=0.3),
                                  A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), p=1.0),
                                  ToTensorV2(),
                                  ])
        
    return test_transform

In [ ]:
class PytorchTrainer :
  @inject_config
  def __init__(self, config:CN) :
    
    self.model_name  = config.model.name_model
    self.EPOCHS  = config.model.epochs
    self.lr  = config.model.base_lr
    self.weight_decay = config.model.weight_decay
    self.train_bs = config.model.train_bs
    self.test_bs = config.model.test_bs
    self.LABELS = config.data.LABELS
    self.NTargets = config.num_classes
    self.device   = config.device
    self.SEED_VAL  = config.seed
    self.n_splits  = config.n_folds
    self.metric = config.metric
  
  def get_score(self,y_true, y_pred):
    return accuracy_score(y_true, np.argmax(y_pred,axis=1))

  def seed_all(self):
        random.seed(self.SEED_VAL)
        np.random.seed(self.SEED_VAL)
        torch.manual_seed(self.SEED_VAL)
        torch.cuda.manual_seed_all(self.SEED_VAL)
        os.environ['PYTHONHASHSEED'] = str(self.SEED_VAL)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False 

  def kfold_split(self,Train,y):
        Train["folds"]=-1
        kf = StratifiedKFold(n_splits= self.n_splits,random_state=self.SEED_VAL,shuffle=True)
        for fold, (_, val_index) in enumerate(kf.split(Train,y)):
                Train.loc[val_index, "folds"] = fold
        return Train
  
  def Train_One_Fold(self,train_df , valid_df,fold):
    self.seed_all()
    trainer = Trainer()
    augs = Augmentation()

    train_transform = augs.train_transform()
    val_transform   = augs.test_trasnform()

    train_dataset = ImageDataset(train_df.reset_index(),train_transform)
    valid_dataset = ImageDataset(valid_df.reset_index(),val_transform)
    
    train_data_loader = DataLoader(dataset=train_dataset,shuffle=True,batch_size=self.train_bs)
    valid_data_loader = DataLoader(dataset=valid_dataset,shuffle=False,batch_size=self.test_bs)
    
    if 'eff' in self.model_name or 'resnest' in self.model_name or 'resnet200d' in self.model_name : 
        model = TimmModel(model_name=self.model_name) 
        model.to(self.device)
    elif 'swin' in self.model_name or 'vit' in self.model_name: 
        model = SwinModel(model_name=self.model_name) 
        model.to(self.device)
    else :
        model = TrochvisionModel(model_name=self.model_name) 
        model.to(self.device)

    optimizer = torch.optim.Adam(model.parameters(), lr=self.lr, weight_decay=self.weight_decay)
    best_score = np.inf
    for epoch in range(self.EPOCHS):
      print("----------------EPOCH "+str(epoch+1)+"---------------------")
      Acc_train = trainer.train_fn(train_data_loader, model, optimizer,scheduler=None)
      loss_val,Score_val,oof_ = trainer.eval_fn(valid_data_loader ,model)
      if loss_val<best_score:
        best_score = loss_val 
        oof_1f = np.copy(oof_) 
        torch.save(model.state_dict(),f"best_model_{fold}") 
    return oof_1f , best_score 

  def Train_K_folds(self,train):
    self.seed_all()
    oof = np.zeros((train.shape[0],self.NTargets))
    train = self.kfold_split(train,train[self.LABELS])
    
    for fold in range(self.n_splits):
      print(f"#########################  Fold {fold+1}/{self.n_splits}  #########################")
      train_df = train[train['folds']!=fold]
      valid_df = train[train['folds']==fold]
      oof_1f , best_score = self.Train_One_Fold(train_df , valid_df,fold)
      oof[valid_df.index.tolist()] = oof_1f 
      free_memory(sleep_time=0.1)
    print(f'{self.metric} OOF Score :',self.get_score(train[self.LABELS],oof))
    return oof

  def InferenceValid(self,valid_df,Augstype='Resize') :
    self.seed_all()
    print(f'Inference On {Augstype} Augs ...')
    augs = Augmentation()
    test_transform   = augs.test_trasnform(Augstype=Augstype)
    valid_dataset = ImageDataset(valid_df.reset_index(drop=True),test_transform,mode='test')
    test_data_loader = DataLoader(dataset=valid_dataset,shuffle=False,batch_size=self.test_bs)
    if 'eff' in self.model_name or 'resnest' in self.model_name or 'resnet200d' in self.model_name :  
        best_model = TimmModel(model_name=self.model_name) 
    elif 'swin' in self.model_name or 'vit' in self.model_name: 
        best_model = SwinModel(model_name=self.model_name) 
    else :
        best_model = TrochvisionModel(model_name=self.model_name) 
    best_model.load_state_dict(torch.load(f'best_model_{fold}'))
    best_model.to(self.device)
    best_model.eval()
    
    final_outputs = []
    with torch.no_grad() :
        tk0 = tqdm(test_data_loader, total=len(test_data_loader))
        for bi,d in enumerate(tk0):
          images = d['image']
          #send them to device 
          images = images.to(self.device,dtype=torch.float)
          outputs  = best_model(images)
          final_outputs.append(torch.nn.functional.softmax(outputs).cpu().detach().numpy())

    final_outputs = np.concatenate(final_outputs)
    Final_Prediction   = np.argmax(final_outputs,axis=1)
    return  final_outputs , Final_Prediction 

  def Inference(self,test,Augstype='Resize') :
    augs = Augmentation()
    test_transform   = augs.test_trasnform(Augstype=Augstype)

    test_dataset = ImageDataset(test.reset_index(drop=True),test_transform,mode='test')
    test_data_loader = DataLoader(dataset=test_dataset,shuffle=False,batch_size=self.test_bs)

    all_folds = []
    for fold in range(self.n_splits):
      print(f"#########################  Fold {fold+1}/{self.n_splits}  #########################")
      if 'eff' in self.model_name or 'resnest' in self.model_name  or 'resnet200d' in self.model_name : 
        best_model = TimmModel(model_name=self.model_name) 
      elif 'swin' in self.model_name or 'vit' in self.model_name: 
          best_model = SwinModel(model_name=self.model_name) 
      else :
          best_model = TrochvisionModel(model_name=self.model_name) 
        
      best_model.load_state_dict(torch.load(f'best_model_{fold}'))
      best_model.to(self.device)
      best_model.eval()
      final_outputs = []
      with torch.no_grad() :
        tk0 = tqdm(test_data_loader, total=len(test_data_loader))
        for bi,d in enumerate(tk0):
          images = d['image']
          #send them to device 
          images = images.to(self.device,dtype=torch.float)
          outputs  = best_model(images)
          final_outputs.append(F.softmax(outputs, dim=1).data.squeeze().cpu().detach().numpy())

      final_outputs = np.concatenate(final_outputs)
      all_folds.append(final_outputs)
    Final_Prediction_f = np.mean(all_folds,axis=0)
    Final_Prediction   = np.argmax(Final_Prediction_f,axis=1)
    return  Final_Prediction_f , Final_Prediction

In [ ]:
AssazzinTrainer = PytorchTrainer()

# **Training**

In [ ]:
import gc ; gc.collect()

673

In [ ]:
import gc
import time
def free_memory(sleep_time=0.1) :
    """ Black magic function to free torch memory and some jupyter whims """
    gc.collect()
    torch.cuda.synchronize()
    gc.collect()
    torch.cuda.empty_cache()
    time.sleep(sleep_time)

free_memory(sleep_time=0.1)

In [ ]:
OOF = AssazzinTrainer.Train_K_folds(train=Train)

#########################  Fold 1/10  #########################


Downloading: "https://github.com/SwinTransformer/storage/releases/download/v1.0.0/swin_base_patch4_window12_384_22kto1k.pth" to /root/.cache/torch/hub/checkpoints/swin_base_patch4_window12_384_22kto1k.pth


----------------EPOCH 1---------------------


  0%|          | 0/469 [00:00<?, ?it/s]

Training loss for this epoch:  0.7139921058088478
Training F1 Score for this epoch:  0.9166666666666666
Validation loss for this epoch:  0.16656436445191503
Validation F1 Score for this epoch 0.9438070168459144
----------------EPOCH 2---------------------


...

Training loss for this epoch:  0.22394283881226282
Training F1 Score for this epoch:  1.0
Validation loss for this epoch:  0.16590145053341984
Validation F1 Score for this epoch 0.9581863578646546
----------------EPOCH 6---------------------


  0%|          | 0/469 [00:00<?, ?it/s]

Training loss for this epoch:  0.20805580705254853
Training F1 Score for this epoch:  1.0
Validation loss for this epoch:  0.1916044836398214
Validation F1 Score for this epoch 0.9520363842832982
----------------EPOCH 7---------------------


  0%|          | 0/469 [00:00<?, ?it/s]

Training loss for this epoch:  0.18727110302442365
Training F1 Score for this epoch:  1.0
Validation loss for this epoch:  0.17297794260084629
Validation F1 Score for this epoch 0.9502448993569149
LogLoss OOF Score : 0.9495919347095535


# Predicting

In [ ]:
free_memory(sleep_time=1.0)

In [ ]:
Final_Prediction_f1 , _ = AssazzinTrainer.Inference(test=Test,Augstype ='Resize')
print('-----------')
Final_Prediction_f2 , _ = AssazzinTrainer.Inference(test=Test,Augstype ='Center_Crop_Resize')
print('-----------')
Final_Prediction_f3 , _ = AssazzinTrainer.Inference(test=Test,Augstype ='RandomCrop')
print('-----------')
Final_Prediction_f4 , _ = AssazzinTrainer.Inference(test=Test,Augstype ='RRR_PROCESS')

In [ ]:
from scipy.stats.mstats import gmean

In [ ]:
def weighted_gmean(weights,predictions) :
  weighted_gmean = 0
  sum_1 =0
  sum_2 =0
  for i in range(len(weights)) :
    sum_1 += weights[i]* np.log(predictions[i])
    sum_2 += weights[i]
  weighted_gmean += np.exp(sum_1/sum_2)
  return weighted_gmean

In [ ]:
test_dict = {'Original': Final_Prediction_f1, 
            'CenterCrop': Final_Prediction_f2, 
            'RandomCrop': Final_Prediction_f3, 
            'Koktel': Final_Prediction_f4, 
           }
BlendTest = np.zeros((len(test_dict), Test.shape[0],y_true.shape[1]))
for i in range(BlendTest.shape[0]):
    BlendTest[i] = np.array(list(test_dict.values())[i])

In [ ]:
GmeanPreds = weighted_gmean([0.440908710790076, 0.20999217856819247, 0.16915344429528964, 0.17994566634644196],BlendTest)

In [ ]:
np.save(f'AssazzinBaseline_{c.model.name_model}_GmeanPreds_TTAx4.npy',GmeanPreds)